In [1]:
import os
os.chdir("../")
%pwd

'd:\\machine-lerning\\Chest-Cancer-DeepLearning'

In [4]:
!pip install dagshub

  Using cached dagshub-0.5.8-py3-none-any.whl.metadata (12 kB)
  Using cached dacite-1.6.0-py3-none-any.whl.metadata (14 kB)
  Using cached gql-3.5.1-py2.py3-none-any.whl.metadata (9.4 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached treelib-1.7.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached pathvalidate-3.2.3-py3-none-any.whl.metadata (12 kB)
  Using cached boto3-1.37.2-py3-none-any.whl.metadata (6.6 kB)
  Using cached dagshub_annotation_converter-0.1.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached lxml-5.3.1-cp312-cp312-win_amd64.whl.metadata (3.8 kB)
  Using cached botocore-1.37.2-py3-none-any.whl.metadata (5.7 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.11.3-py3-none-any.whl.metadata (1.7 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached graphql_core-3.2.3-py3-none-any.whl

In [5]:
import dagshub
dagshub.init(repo_owner='irshadmuhammed', repo_name='Chest-Cancer-DeepLearning', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

d:\machine-lerning\Chest-Cancer-DeepLearning\venv\Lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=6483c1f8-dea8-45ae-a3f0-c7da3ba73a16&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=9c72fc3e7ca316d2c8edd7b3fcfbfa07354c8afecdccf7937c20e61931e7d6cd




Accessing as irshadmuhammed

Initialized MLflow to track repo "irshadmuhammed/Chest-Cancer-DeepLearning"

Repository irshadmuhammed/Chest-Cancer-DeepLearning initialized!

🏃 View run valuable-auk-139 at: https://dagshub.com/irshadmuhammed/Chest-Cancer-DeepLearning.mlflow/#/experiments/0/runs/0b53cc9da8e544459169569ef5c372ee
🧪 View experiment at: https://dagshub.com/irshadmuhammed/Chest-Cancer-DeepLearning.mlflow/#/experiments/0


In [ ]:
set MLFLOW_TRACKING_USERNAME=irshadmuhammed
set MLFLOW_TRACKING_PASSWORD=57152f545d8a0301d0b235fa4a218a9877392908



In [6]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from ChestCancerDetection.constants import *
from ChestCancerDetection.utils.common import read_yaml, create_directories, save_json
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Data",
            mlflow_uri="https://dagshub.com/irshadmuhammed/Chest-Cancer-DeepLearning.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [9]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [10]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 139 images belonging to 2 classes.


d:\machine-lerning\Chest-Cancer-DeepLearning\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.9946 - loss: 0.0187


2025/02/27 13:48:23 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/02/27 13:48:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'VGG16Model'.
2025/02/27 13:49:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


🏃 View run spiffy-cod-892 at: https://dagshub.com/irshadmuhammed/Chest-Cancer-DeepLearning.mlflow/#/experiments/0/runs/a13d4324efb34996af52ed8b6927c5c2
🧪 View experiment at: https://dagshub.com/irshadmuhammed/Chest-Cancer-DeepLearning.mlflow/#/experiments/0
